Задание №6. Построение карт энтропии

In [34]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [35]:
# Чтение и обработка данных
data = open('input/facie_cube').read().splitlines()[5:]
data = pd.DataFrame([x.split(sep= ' ') for x in data]).drop(columns=3).astype(float)
data.columns = ['i','j','k','f']

# Конвертация индексов к нулевой базе
data['i'] = data['i'] - 1
data['j'] = data['j'] - 1
data['k'] = data['k'] - 1

In [36]:
# Создание куба для хранения данных о литологии
i_max, j_max, k_max = data[['i', 'j', 'k']].max().astype(int) + 1
cube = np.zeros((i_max, j_max, k_max))

# Заполнение куба данными о литологии
for index, row in data.iterrows():
    i, j, k, f = row
    cube[int(i), int(j), int(k)] = f

In [37]:
# Создание тензора для хранения векторов вероятности
prob_tensor = np.zeros((i_max, j_max, k_max, 5))  # предполагаем 5 уникальных литологий

# Вычисление вероятностей для каждой ячейки i, j по всем слоям k
for i in range(i_max):
    for j in range(j_max):
        lithologies = cube[i, j, :].astype(int)
        counts = Counter(lithologies)
        for lithology, count in counts.items():
            if lithology >= 0:  # игнорируем литологии равные -1
                prob_tensor[i, j, :, lithology] = count / k_max

In [38]:
# Вычисление энтропии
entropy_tensor = np.zeros((i_max, j_max, k_max))
for i in range(i_max):
    for j in range(j_max):
        for k in range(k_max):
            probabilities = prob_tensor[i, j, k, :]
            entropy_tensor[i, j, k] = entropy(probabilities)

In [43]:
# Усреднение энтропии по слоям k для получения двухмерного DataFrame
entropy_df = pd.DataFrame(entropy_tensor.mean(axis=2))

fig = px.imshow(entropy_df, text_auto=True, aspect="auto")
fig.show()